In [14]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SequentialFeatureSelector
from datetime import datetime


In [15]:
df_train = pd.read_csv('../test_ml/main_df.csv')
df_test = pd.read_csv('../test_ml/main_comp.csv')

""" 
transactions_df = pd.read_csv('../test_ml/transactions_df.csv')
irrelevant_columns_df = pd.read_csv('../test_ml/irrelevant_columns_df.csv')
amount_outliers_df = pd.read_csv('../test_ml/amount_outliers_df.csv')
age_loan_outliers_df = pd.read_csv('../test_ml/age_loan_outliers_df.csv')
"""

" \ntransactions_df = pd.read_csv('../test_ml/transactions_df.csv')\nirrelevant_columns_df = pd.read_csv('../test_ml/irrelevant_columns_df.csv')\namount_outliers_df = pd.read_csv('../test_ml/amount_outliers_df.csv')\nage_loan_outliers_df = pd.read_csv('../test_ml/age_loan_outliers_df.csv')\n"

In [16]:
X_train = df_train.drop(columns='status', axis=1)
y_train = df_train[['status']]

X_test = df_test.drop(columns='status', axis=1)

In [17]:
pipeline = []
model_instance = RandomForestClassifier(criterion='gini', max_depth=10, n_estimators=50, n_jobs=-1)
rfe = SequentialFeatureSelector(model_instance, n_features_to_select="auto", tol=0.07)

pipeline.append(('rfe', rfe))
pipeline.append(("model", model_instance))

pipe = Pipeline(steps=pipeline)

In [18]:
pipe.fit(X_train, y_train)
y_result = pipe.predict_proba(X_test)[:, 0]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks

In [19]:
result = pd.DataFrame({"Id": X_test["loan_id"], "Predicted": y_result})
result.drop_duplicates(inplace=True)
time = datetime.now().strftime('%d_%H-%M-%S')
result.to_csv(f"./results/result-{time}.csv", index=False)
with open(f"./results/result-{time}.txt", 'w') as f:
    f.write(f"Random Forest\nFeature Selection\n{model_instance.get_params()}")